# PyLadies Data Analysis Lab @ Flatiron Health, May 2019

Workshop lead: Sam Bail, Twitter @spbail

# Overview
The goal of this workshop is to give learners an intro to data analysis with Python using Pandas and Jupyter. 
We will first go through the process of loading data from CSV files, inspecting and cleaning the data. As a second step, we will analyse the data and draw some insights about cancer treatment from it. 

The tutorial is structured as follows:

- Part 0: Setup
- Part 1: Loading and inspecting data
- Part 2: Data cleaning
- Part 3: Data analysis
- Part 4: Summary

**Note that this tutorial is only intended as an introduction to some basic concepts of Pandas. It is in no means intended to be comprehensive, and there are a lot of useful functions a beginner needs to know to do in-depth data analysis. We hope that this tutorial sets you up for self-guided learning to master the full range of necessary Pandas tools.**

# Part 0: Setup

Make sure you have the following up and running on your laptop:
- A working python 3 environment
- Working installations of jupyter, pandas, matplotlib, seaborn (use `pip install` to install them)
- With your python environment activated, open a terminal and run `jupyter notebook` to start up the notebook server
- Then go to "New" to create a new notebook and you're ready to start the tutorial!

## Imports

To complete this assignment we will use the Pandas data analysis library, with the Seaborn library for visualization.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This command makes charts show inline in a notebook
%matplotlib inline

In [2]:
import sys
print("Python Version:", sys.version)

Python Version: 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
import matplotlib

In [4]:
matplotlib.__version__

'2.2.2'

In [5]:
pd.__version__

'0.23.0'

In [6]:
sns.__version__

'0.8.1'

## What is Pandas/Matplotlib/Pyplot/Seaborn

- Pandas is a Python library for data manipulation and analysis. It offers data structures and operations for manipulating numerical tables and time series.
- Matplotlib is a Python 2D plotting library. Pyplot is a collection of command style functions that make matplotlib work like MATLAB. While we mostly use Seaborn, we sometimes fall back to using Pyplot functions for certain aspects of plotting.
- Seaborn is a Python data visualization library based on matplotlib. It's kind of like a nicer version of pyplot.


# Part 1: Loading and inspecting the data

Before we can start answering questions about the data we need to do a little bit of exploratory analysis.The first thing we need to do when working with a new dataset is to get an idea of what the data looks like. We start by loading the data into memory. Pandas comes with a built-in `read_csv` function that we can use to read CSV files and load them directly to a pandas `DataFrame` object. 

In [ ]:
tx = pd.read_csv('./treatment_starts_2016.csv')

## What is a DataFrame?
A DataFrame is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects. It is generally the most commonly used Pandas object. 

Pandas borrows the concept of DataFrame from the statistical programming language R.

Let's take a look at the data to familiarize ourselves with the format and data types.

In [ ]:
# Just using the name of the dataframe will print the entire output
# If there are two many rows, Jupyter will print the top few and bottom few rows
tx

In [ ]:
# The head() function shows the first n rows in a dataframe.
tx.head()

In [ ]:
# You can also use the sample() function to get n random rows in the dataframe
# NOTE: sample() only works in newer versions of pandas (0.16.1 and upwards)
tx.sample(3)

In [ ]:
# Then len function gives us the number of rows in the dataframe
len(tx)

In [ ]:
# The dtypes property of a dataframe shows the datatypes of every column in a dataframe.
tx.dtypes

## Accessing columns in a dataframe

<span style="color:blue">**Note: We will be applying `head()` to some results in this tutorial to keep the output short. When working with a real dataset, keep in mind that you might be hiding some interesting records if you always use `head()` or `sample()`!**</span>

In Pandas, you can access a specific column using the following notation which returns a **Series** (not a dataframe) - a Series is simply a vector, aka a 1-dimensional data structure similar to a list.

In [ ]:
tx.PatientID.head()

In [ ]:
# Check the type to show that this indeed returns a Series object
type(tx.PatientID)

In [ ]:
# The alternative notation for accessing a column in a dataframe
# Some people prefer the . notation, others the [] notation.
tx['PatientID'].head()

In [ ]:
# And this is how you access two columns of a dataframe.
# Note that this will return a dataframe again, not a series (because a series has only one column...)
# Also note the double square brackets - you're passing a *list* as an argument
tx[['PatientID', 'TreatmentStart']].head()

In [ ]:
# This way we can now do some more data exploration, e.g. getting unique patients
tx['PatientID'].unique()

In [ ]:
# Or unique drugs
tx['Drug'].unique()

## Accessing rows in a dataframe
In addition to slicing by column, we often want to get the record where a column has a specific value, e.g. a specific Patient_ID here. This can be done using the following syntax:

In [ ]:
# Access the diagnosis record(s) for a specific patient ID
tx.loc[tx['PatientID'] == 'PT5']

In [ ]:
# This is equivalent with the following shorter notation
tx[tx['PatientID'] == 'PT20']

There are many different ways to access rows and columns in Pandas. We're only introducing a small set here in order to keep the tutorial simple.

## Sorting dataframes
Sorting a dataframe by one or multiple columns is super easy:

In [ ]:
# Sort by earliest treatment start date, i.e. in ascending order
tx.sort_values('TreatmentStart').head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort('TreatmentStart').head()

In [ ]:
# Sort by latest treatment start, i.e. in descending order
tx.sort_values('TreatmentStart', ascending=False).head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort('TreatmentStart', ascending=False).head()

In [ ]:
# Finally, you can also sort by multiple columns:
tx.sort_values(['PatientID', 'TreatmentStart']).head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort(['PatientID', 'TreatmentStart']).head()

<span style="color:blue">**Note: Any operations on a dataframe are *not* permanent, i.e. they only modify the current output, but not the actual dataframe. If you want to preserve the sorting, for example, you have to either assign the output to a new variable, or use the `inplace=True` argument. This will not create any output but actually modify the dataframe.**</span>

In [ ]:
tx.sort_values(['PatientID', 'TreatmentStart'], inplace=True)

# Part 2: Data cleaning

In [ ]:
# Remember the dtypes property... the TreatmentStart column should really be a date, right?
tx.dtypes

## Date conversion

Right away we can see that the date field TreatmentDate is stored as string (object). It might be useful to convert it to **Datetime** objects so that we can perform common date arithmetic on them, like checking if a date came before or after another date, or calculating the number of days between two dates.

In [ ]:
tx.TreatmentStart = pd.to_datetime(tx.TreatmentStart)

In [ ]:
tx.dtypes

In [ ]:
# This is the alternative notation to assign a value to a column in a dataframe
tx['TreatmentStart'] = pd.to_datetime(tx.TreatmentStart)

# Part 3: Data analysis
Let's assume we've loaded the treatment related data from a cancer clinic in order to provide them with some analytical insights around the types of drugs they use on their patient population.

## Question 1: Patients treated at the practice

**How many patients does the practice treat?**

In [ ]:
# Our data frame contains patient IDs and treatment starts -
# let's check if some patients have multiple treatment starts?
# The unique() function returns the number of unique values in a dataframe column.
print('Number of treatment start records:', len(tx))
print('Number of unique patients who start treatment:', len(tx.PatientID.unique()))

So there are 20 unique patients but we have 23 treatment starts, meaning some patients start different treatments in the time that we have data for. This means that if we want to answer the question correctly, we need to make sure to only count unique patients.

In [ ]:
# Let's count how often each patient occurs in the data
tx.groupby('PatientID').count()[['TreatmentStart']]

# This shows us that 3 patients in the data have 2 treatment starts each
# Since patients can switch treatment, this is not necessarily a bug!

## Question 2: Drugs used at the practice
**What are the drugs used at the practice and how many patients receive those drugs?**

In [ ]:
# The groupby function works like a groupby in SQL, i.e. it groups the dataframe by the specified
# column and then lets you apply aggregate functions on the grouped values, e.g. counts, sums, means...
# The count function counts the number of rows with values in a column
# NOTE that this might include duplicates!!! (Not in this data...)
tx.groupby('Drug').count()

In [ ]:
# Since we are only interested in the number of patients, 
# we select only the relevant column from the resulting dataframe
# Note that "PatientID" might not be the best name for this column
# - we can use a rename() function in Pandas to rename it to something like
# "PatientCount" (skipping the rename step in this tutorial, but feel free to look it up!)
tx.groupby('Drug').count()[['PatientID']]

### A little bit about indexes in dataframes
Notice that in the above example, the "Drug" column is printed in bold. That's because grouping by it has turned it into the **index** of the resulting dataframe.

The index in a dataframe is the "row identifier" - it is generally printed as the column on the left. For example, when we first loaded our data, the index didn't have a name and was just an incrementing integer (scroll up to check!). When you create a groupby object, the index of a resulting dataframe will be the column you group by - in this case, the Drug column became the index.

We frequently **reset** the index in a dataframe for various reasons - in this case, because the index contains data that you want to treat as a column, e.g. for plotting.

In [ ]:
# Reset the index in this resulting dataframe to see what happens:
tx.groupby('Drug').count()[['PatientID']].reset_index()

In [ ]:
# Remember that any operations on the dataframe only modify the output? 
# We didn't *really* group tx or reset the index. 
# The tx dataframe is still the same it was at the beginning:
tx.head()

### Let's plot this! (aka our first Seaborn plot)

In [ ]:
# Let's do the same groupby as above to get the number of patient starts per drug!
# This time, we actually assign the output to a new dataframe to make the transformation permanent!
counts = tx.groupby('Drug').count()[['PatientID']].reset_index()

# Let's use a simple bar chart in Seaborn to compare counts for the two drugs
# There are several different ways to do the plotting - this is my preferred style,
# but you might prefer different syntax
fig = sns.barplot(data=counts, x='Drug', y='PatientID')
plt.title('Number of patients by drug')
plt.ylabel('Number of patients')
plt.xlabel('Drug')
plt.show(fig)

## Question 3: Changes to treatment over time
**Do we see any changes in treatment patterns over time?**

Our data shows treatment starts by date. Let's group these starts by month to see if there are any changes of how many patients start on a given drug over time, e.g. because a new drug got approved.

*Note that the data we're using here is dummy data and pretty artificial - oncology clinics see a much higher volume of patients, and drug uptake is usually slower than shown here.*

In [ ]:
# Let's add a new column that only has the treatment month to simplify things
# There are many different ways to do this, we picked a simple one
# tx['TreatmentStartMonth'] = tx['TreatmentStart'].astype('datetime64[M]')

# NOTE .astype('datetime64[M]') only works in more recent versions of Pandas, this is an older version:
from datetime import datetime
tx['TreatmentStartMonth'] = tx['TreatmentStart'].apply(lambda x: x.replace(day=1))

tx.head()

In [ ]:
# Let's count the number of starts per month per drug to plot it later
# We only want the number of patients, so we filter for that column at the end
drugs_by_month = tx.groupby(['TreatmentStartMonth', 'Drug']).count()[['PatientID']]
drugs_by_month

In [ ]:
# The data already looks interesting... let's plot this 
# Remember to reset_index so we can plot the regular columns
# The "hue" keyword is generally used to distinguish two different categorial variables
# in plots, e.g. in this case the two different drugs
# NOTE: lineplot() only exists in Seaborn version 0.9 and up
fig = sns.lineplot(data=drugs_by_month.reset_index(), x='TreatmentStartMonth', y='PatientID', hue='Drug')

# Part 4: Summary!

We hope this workshop was useful for you. We've only touched on some of the basic concepts of pandas, but we believe this will give you the foundations to keep exploring the data! We covered:

- Dataframes and series in pandas
- Basic data inspection (head, describe, dtypes, accessing columns and rows, sorting)
- Grouping and aggregating (count, nunique)
- Indexing in dataframes and reset_index
- Plotting (bar plots, line plots)

**What we didn't learn:**

This is my (biased) list of very frequent Pandas operations that we didn't cover but you'll likely need for data analysis:
- Joining/merging multiple dataframes
- Filtering and de-duplicating dataframes
- More complex modifications of column values, e.g. filling null values, using lambda functions
- More complex aggregates on grouped dataframes (sum, mean, etc)
- Renaming columns (e.g. renaming an aggregate "PatientID" column to something more meaningful like "PatientCount")